# PFS - experimental support

Support for PFS is limited to files with unique subsections. This applies to all files in the MIKE FM family.

It is only possible to read files, there is no support for modifying or writing files at this stage.

In [1]:
from mikeio import Pfs

pfs = Pfs("../tests/testdata/lake.sw")

In [2]:
pfs.data.MODULE_SELECTION.mode_of_spectral_wave_module

2

In [3]:
pfs.data.SW.SPECTRAL.number_of_directions

16

In [4]:
pfs.data.SW.SPECTRAL.maximum_threshold_frequency

0.5959088268863615

In [5]:
df = pfs.data.SW.get_outputs()
df

,file_name,include,type,format,first_time_step,last_time_step,use_end_time,time_step_frequency
0,Wave_parameters.dfsu,1,1,2,0,450,1,10
1,Wave_line.dfs1,0,1,1,0,450,1,10
2,Waves_x20km_y20km.dfs0,1,1,0,0,450,1,1
3,spectrum_x20km_y20km.dfsu,1,4,0,0,450,1,10
